# Import needed libraries

In [10]:
%pip install python-chess

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import chess
import chess.pgn
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Feature engineering

In [13]:
# Function to generate features from a FEN
def generate_features(fen):
    board = chess.Board(fen)
    features = np.zeros((64, 13), dtype=np.uint8)

    for i in range(64):
        if board.piece_at(i) is not None:
            piece = board.piece_at(i).symbol()
            index = ' PNBRQKpnbrqk'.index(piece)
            features[i][index] = 1

    return features.flatten()

## Option 1

In [14]:
# Create training data
training_data = []
training_labels = []

# Load chess games from a PGN file
with open('/content/drive/MyDrive/Saturdays/games.pgn') as file:
    while True:
        game = chess.pgn.read_game(file)
        if game is None:
            break

        board = game.board()
        tactical_themes = ['Checkmate', 'Double Attack', 'Deflection']

        for move in game.mainline_moves():

            fen = board.fen()
            features = generate_features(fen)

            # Generate label based on tactical theme
            label = 0
            if board.is_checkmate():
                label = 1
            elif board.is_check() or board.is_attacked_by(board.turn, board.king(not board.turn)):
                label = 2
            print(label)
            print(fen)
            training_data.append(features)
            training_labels.append(label)

            board.push(move)

# Convert data to NumPy arrays
training_data = np.array(training_data)
training_labels = np.array(training_labels)


Se han truncado las últimas 5000 líneas del flujo de salida.
0
r1bqkb1r/pppp1ppp/2n2n2/4p3/8/1P2PN2/PBPP1PPP/RN1QKB1R b KQkq - 2 4
0
r1bqk2r/pppp1ppp/2nb1n2/4p3/8/1P2PN2/PBPP1PPP/RN1QKB1R w KQkq - 3 5
0
r1bqk2r/pppp1ppp/2nb1n2/4p3/2P5/1P2PN2/PB1P1PPP/RN1QKB1R b KQkq - 0 5
0
r1bq1rk1/pppp1ppp/2nb1n2/4p3/2P5/1P2PN2/PB1P1PPP/RN1QKB1R w KQ - 1 6
0
r1bq1rk1/pppp1ppp/2nb1n2/4p3/2P5/1P1PPN2/PB3PPP/RN1QKB1R b KQ - 0 6
0
r1bqr1k1/pppp1ppp/2nb1n2/4p3/2P5/1P1PPN2/PB3PPP/RN1QKB1R w KQ - 1 7
0
r1bqr1k1/pppp1ppp/2nb1n2/4p3/2P5/PP1PPN2/1B3PPP/RN1QKB1R b KQ - 0 7
0
r1bqr1k1/1ppp1ppp/2nb1n2/p3p3/2P5/PP1PPN2/1B3PPP/RN1QKB1R w KQ - 0 8
0
r1bqr1k1/1ppp1ppp/2nb1n2/p3p3/2P5/PP1PPN2/1B2BPPP/RN1QK2R b KQ - 1 8
0
r1bqrbk1/1ppp1ppp/2n2n2/p3p3/2P5/PP1PPN2/1B2BPPP/RN1QK2R w KQ - 2 9
0
r1bqrbk1/1ppp1ppp/2n2n2/p3p3/2P5/PP1PPN2/1B2BPPP/RN1Q1RK1 b - - 3 9
0
r1bqrbk1/1pp2ppp/2n2n2/p2pp3/2P5/PP1PPN2/1B2BPPP/RN1Q1RK1 w - - 0 10
0
r1bqrbk1/1pp2ppp/2n2n2/p2Pp3/8/PP1PPN2/1B2BPPP/RN1Q1RK1 b - - 0 10
0
r1bqrbk1/1pp2ppp/2n5/p

## Option 2

In [15]:
# Create training data
training_data = []
training_labels = []

# Load chess games from a PGN file
with open('/content/drive/MyDrive/Saturdays/games.pgn') as file:
    while True:
        game = chess.pgn.read_game(file)
        if game is None:
            break

        board = game.board()

        # Labels for tactical themes
        checkmate_label = [1, 0, 0]  # Checkmate: [1, 0, 0]
        double_attack_label = [0, 1, 0]  # Double Attack: [0, 1, 0]
        pin_label = [0, 0, 1]  # Pin: [0, 0, 1]

        for move in game.mainline_moves():
            fen = board.fen()
            features = generate_features(fen)

            # Generate label based on tactical theme
            label = []
            if board.is_checkmate():
                label = checkmate_label
            elif board.is_check() or board.is_attacked_by(board.turn, board.king(not board.turn)):
                label = double_attack_label
            else:
                label = pin_label

            training_data.append(features)
            training_labels.append(label)

            board.push(move)

# Convert data to NumPy arrays
training_data = np.array(training_data)
training_labels = np.array(training_labels)

# Training

In [16]:
# Reshape the training data to match the input shape of the model
training_data = training_data.reshape((-1, 832))


# Create the neural network model
model = tf.keras.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(832,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(training_data, training_labels, epochs=10, batch_size=32)


# Saved the trained model
model.save("/content/drive/MyDrive/Saturdays/trained_model.h5")

Epoch 1/10
42857/42857 [==============================] - 205s 5ms/step - loss: 0.1009 - accuracy: 0.9499
Epoch 2/10
42857/42857 [==============================] - 200s 5ms/step - loss: 0.0825 - accuracy: 0.9583
Epoch 3/10
42857/42857 [==============================] - 196s 5ms/step - loss: 0.0714 - accuracy: 0.9637
Epoch 4/10
42857/42857 [==============================] - 201s 5ms/step - loss: 0.0640 - accuracy: 0.9675
Epoch 5/10
42857/42857 [==============================] - 196s 5ms/step - loss: 0.0588 - accuracy: 0.9700
Epoch 6/10
42857/42857 [==============================] - 191s 4ms/step - loss: 0.0549 - accuracy: 0.9721
Epoch 7/10
42857/42857 [==============================] - 190s 4ms/step - loss: 0.0517 - accuracy: 0.9738
Epoch 8/10
42857/42857 [==============================] - 187s 4ms/step - loss: 0.0492 - accuracy: 0.9751
Epoch 9/10
42857/42857 [==============================] - 187s 4ms/step - loss: 0.0471 - accuracy: 0.9762
Epoch 10/10
42857/42857 [=====================

In [17]:
# Load the model
model = tf.keras.models.load_model("/content/drive/MyDrive/Saturdays/trained_model.h5")

# Evaluation

## Option 1

In [18]:
# Example of using the model for prediction
example_fen = '4k3/8/8/3R2p1/3p2Bp/3n1P1P/8/1Kr5 w - - 12 56'
example_features = generate_features(example_fen)
example_features = np.array([example_features])
predictions = model.predict(example_features)
predicted_tactical_theme = tactical_themes[np.argmax(predictions)]

print("Predicted Tactical Theme:", predicted_tactical_theme)

1/1 [==============================] - 0s 91ms/step
Predicted Tactical Theme: Double Attack


## Option 2

In [25]:
# Example of using the model for prediction
example_fen = 'rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2'
example_features = generate_features(example_fen)
example_features = np.array([example_features])
predictions = model.predict(example_features)

# Decode the predicted labels
checkmate_pred = predictions[0][0]
double_attack_pred = predictions[0][1]
pin_pred = predictions[0][2]

print("Checkmate prediction:", checkmate_pred)
print("Double Attack prediction:", double_attack_pred)
print("Pin prediction:", pin_pred)


1/1 [==============================] - 0s 33ms/step
Checkmate prediction: 0.0
Double Attack prediction: 3.4218213e-19
Pin prediction: 1.0


In [26]:
#fen = 'rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2'
fen = '4k3/8/8/3R2p1/3p2Bp/3n1P1P/8/1Kr5 w - - 12 56'
board = chess.Board(fen)

print(board.is_checkmate())
print(board.is_check())
print(board.is_attacked_by(board.turn, board.king(not board.turn)))

False
True
False


In [30]:
fen = '4k3/8/8/3R2p1/3p2Bp/3n1P1P/8/1Kr5 w - - 12 56'
board = chess.Board(fen)

if board.is_check():
    print("Hay un jaque en la posición.")
else:
    print("No hay un jaque en la posición.")

Hay un jaque en la posición.
